In [1]:
import utils
from utils import *
import xgboost as xgb

%matplotlib inline


/home/ubuntu/anaconda2/envs/qenv/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Helper classes 

### Load features

In [3]:
q1_data = np.load(open(Q1_TRAINING_DATA_FILE, 'rb'))
q2_data = np.load(open(Q2_TRAINING_DATA_FILE, 'rb'))


In [17]:
abhishek_train = pd.read_csv(data_home+"abhishek_train_features.csv")
abhishek_test = pd.read_csv(data_home+"abhishek_test_features.csv")

In [18]:
def get_and_normalize_features(df, feature_list):
    
    np_version = np.array(df.ix[:,feature_list])
#     np_version = np.nan_to_num(np_version)
#     np_version = normalize(np_version ,axis=0,norm='max')
    
    return np_version

In [12]:
# q1_features = [u'len_q1', u'len_char_q1', u'len_word_q1', u'skew_q1vec', u'kur_q1vec']
# q2_features = [u'len_q2', u'len_char_q2', u'len_word_q2', u'skew_q2vec', u'kur_q2vec']
# diff_features = abhishek_train.columns.difference(q1_features).difference(q2_features).difference(['question1','question2'])

In [17]:
# abhishek_q1_features_np = get_and_normalize_features(abhishek_train,q1_features)
# abhishek_q2_features_np = get_and_normalize_features(abhishek_train,q2_features)
# abhishek_diff_features_np = get_and_normalize_features(abhishek_train,diff_features)

In [20]:
# abhishek_q1_features_np.shape,abhishek_q2_features_np.shape,abhishek_diff_features_np.shape

In [21]:
abhishek_train = abhishek_train.ix[:,2:]

In [22]:
abhishek_test = abhishek_test.ix[:, 2:] 

In [23]:
abhishek_train.shape,abhishek_test.shape

((404290, 28), (2345796, 28))

#### Magic feature!

In [26]:
q_frequency_train = pd.read_csv("data/question_frequency_train.csv")
q_frequency_test = pd.read_csv("data/question_frequency_test.csv")


In [27]:
q_frequency_train.shape, q_frequency_test.shape

((404290, 6), (2345796, 5))

In [30]:
q_frequency_train  = q_frequency_train.ix[:,3:5]

In [31]:
q_frequency_test = q_frequency_test.ix[:,3:5]

In [32]:
q_frequency_train.shape, q_frequency_test.shape

((404290, 2), (2345796, 2))

#### Nostrov features

In [35]:
nostrov = pd.read_csv('data/nestrov_feats.csv')

In [36]:
ix_train = np.where(nostrov['id'] >= 0)[0]
ix_test = np.where(nostrov['id'] == -1)[0]

In [37]:
nostrov_diff = nostrov.ix[:,10:]

In [38]:
nostrov_train = nostrov_diff.iloc[ix_train]
nostrov_test = nostrov_diff.iloc[ix_test]

nostrov_train.shape, nostrov_test.shape

((404290, 28), (2345796, 28))

In [39]:
del nostrov, nostrov_diff

#### 2-gram diffs

2gram > 3gram > 2gram-and-3gram  >  ngram(2,3)

In [195]:
twogram_difs = pd.read_csv("data/cache/23gram_distance.csv")

In [196]:
ix_train = np.where(twogram_difs['id'] >= 0)[0]
ix_test = np.where(twogram_difs['id'] == -1)[0]

In [197]:
twogram_difs.columns

Index([u'id', u'is_duplicate', u'qid1', u'qid2', u'question1', u'question2',
       u'test_id', u'23ngram_tfidf_cosine', u'23ngram_tfidf_euclidean',
       u'23ngram_tf_euclidean', u'23ngram_tf_cosine', u'2gram_tfidf_cosine',
       u'2gram_tfidf_euclidean', u'2gram_tf_euclidean', u'2gram_tf_cosine',
       u'3gram_tfidf_cosine', u'3gram_tfidf_euclidean', u'3gram_tf_euclidean',
       u'3gram_tf_cosine'],
      dtype='object')

In [199]:
# #2-3 gram distance 

twogram_difs = twogram_difs.ix[:,-4:]

twogram_train = twogram_difs.iloc[ix_train]
twogram_test = twogram_difs.iloc[ix_test]

twogram_train.shape, twogram_test.shape

((404290, 4), (2345796, 4))

In [100]:
# twogram_difs = twogram_difs.ix[:,-4:]
# twogram_train = twogram_difs.iloc[ix_train]
# twogram_test = twogram_difs.iloc[ix_test]

# twogram_train.shape, twogram_test.shape

((404290, 4), (2345796, 4))

In [125]:
del twogram_difs

#### LSTM features

In [40]:
# lstm_features_train = load_array("data/cache/lstm_features_train_q1_q2.dat/")
lstm_features_train = pd.read_csv("data/cache/lstm_distances_train_q12.csv")

In [41]:
# lstm_features_test = load_array("data/cache/lstm_features_test_q1_q2.dat/")
lstm_features_test = pd.read_csv("data/cache/lstm_distances_test_q12.csv")

In [42]:
lstm_features_train.shape, lstm_features_test.shape

((404290, 7), (2345796, 7))

#### common neighbours feature

In [43]:
common_neighbours_train = pd.read_csv("data/cache/common_neighbours_train.csv")
common_neighbours_test = pd.read_csv("data/cache/common_neighbours_test.csv")


In [44]:
common_neighbours_train.shape, common_neighbours_test.shape

((404290, 1), (2345796, 1))

#### Group features

In [34]:
# same_group_train = pd.read_csv("data/same_group_train.csv")
# same_group_test = pd.read_csv("data/same_group_test.csv")


In [36]:
# same_group_train = same_group_train.ix[:,0].values
# same_group_test = same_group_test.ix[:,0].values

#### max k-core feature

In [3]:
kcore_train = pd.read_csv("data/max_kcore_train.csv")
kcore_test = pd.read_csv("data/max_kcore_test.csv")

In [22]:
kcore_train.shape,kcore_test.shape

((404290, 2), (2345796, 2))

#### word match

In [4]:
wm_train = pd.read_csv("data/cache/word_match_train.csv")
wm_test = pd.read_csv("data/cache/word_match_test.csv")

#### Add all together 

In [47]:
# train_features = np.hstack((abhishek_train,q_frequency_train, nostrov_train,lstm_features_train,common_neighbours_train))
# test_features = np.hstack((abhishek_test,q_frequency_test, nostrov_test,lstm_features_test, common_neighbours_test))
# train_features.shape, test_features.shape

In [52]:
# del abhishek_test, abhishek_train, q_frequency_train,q_frequency_test, nostrov_train, nostrov_test, lstm_features_train,lstm_features_test

In [6]:
# save_array("data/cache/abhishek_qfreq_nostrov_lstm_neighbourcount_3gram_maxcore_train.dat", train_features)
# save_array("data/cache/abhishek_qfreq_nostrov_lstm_neighbourcount_3gram_maxcore_test.dat", test_features)

In [51]:
# train_features = load_array("data/cache/ab_qfreq_nostrov_train.dat")
# test_features = load_array("data/cache/ab_qfreq_nostrov_test.dat")
# train_features = load_array("data/cache/abhishek_qfreq_nostrov_lstm_neighbourcount_3gram_train.dat")
# test_features = load_array("data/cache/abhishek_qfreq_nostrov_lstm_neighbourcount_3gram_test.dat")
train_features = load_array("data/cache/abhishek_qfreq_nostrov_lstm_neighbourcount_3gram_maxcore_train.dat")
test_features = load_array("data/cache/abhishek_qfreq_nostrov_lstm_neighbourcount_3gram_maxcore_test.dat")


In [52]:
labels = np.load(open(LABEL_TRAINING_DATA_FILE, 'rb'))

In [39]:
# train_features = np.hstack((train_features,wm_train))
# test_features = np.hstack((test_features,wm_test))

In [53]:
train_features.shape, test_features.shape

((404290, 72), (2345796, 72))

In [54]:
train_features = np.nan_to_num(train_features)
test_features = np.nan_to_num(test_features)

In [8]:
# from sklearn.preprocessing import normalize

# train_features = normalize(train_features ,axis=0,norm='max')
# test_features = normalize(test_features ,axis=0,norm='max')


#### Model Training

In [8]:
num_folds = 5

xgb_params = {'colsample_bylevel': 0.9086627943049572,
 'colsample_bytree': 0.8662812861700129,
 'eval_metric': 'logloss',
 'gamma': 0.02933179779163947,
 'learning_rate': 0.028244194429306306,
 'max_depth': 9,
 'n_estimators': 3567,
 'objective': 'binary:logistic',
 'reg_alpha': 0.14600953461910307,
 'reg_lambda': 1.0750571669994455,
 'scale_pos_weight': 1,
 'subsample': 0.8097993636153147}


# xgb_params = {
# 'max_depth': 9, 
# 'learning_rate': 0.025,
# 'n_estimators': 3000, 
# 'objective': 'binary:logistic',
#     'eval_metric' :'logloss',
#     'scale_pos_weight': 1,
# #             'gamma': 0.3, 
#         'subsample': 0.75, 
#         'colsample_bytree': 0.75, 
#         'colsample_bylevel': 0.75,
# #         'reg_alpha': 0.2, 
#         'reg_lambda': 1, 
    

# }


In [56]:
np.random.seed(2189)

perm = np.random.permutation(len(train_features))
ix_fit = perm[:int(len(train_features)*(1-0.1))]
ix_valid = perm[int(len(train_features)*(1-0.1)):]

y_val = labels[ix_valid]
y_train = labels[ix_fit]


In [57]:
d_train = xgb.DMatrix(train_features[ix_fit],label= y_train)#, weight=weight_train)
d_valid = xgb.DMatrix(train_features[ix_valid],label= y_val)#,weight=weight_val)



############## TRAINING
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
bst = xgb.train(xgb_params, d_train, 500,  watchlist, early_stopping_rounds=50, verbose_eval=10)#,feval=kappa)

#     xgb_models.append(bst)

[0]	train-logloss:0.677866	valid-logloss:0.678037
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[10]	train-logloss:0.548254	valid-logloss:0.550065
[20]	train-logloss:0.462002	valid-logloss:0.465209
[30]	train-logloss:0.399956	valid-logloss:0.404531
[40]	train-logloss:0.356221	valid-logloss:0.362175
[50]	train-logloss:0.323725	valid-logloss:0.330905
[60]	train-logloss:0.298897	valid-logloss:0.307219
[70]	train-logloss:0.279674	valid-logloss:0.289134
[80]	train-logloss:0.264985	valid-logloss:0.275572
[90]	train-logloss:0.253871	valid-logloss:0.265333
[100]	train-logloss:0.244819	valid-logloss:0.257139
[110]	train-logloss:0.237748	valid-logloss:0.251008
[120]	train-logloss:0.231923	valid-logloss:0.245993
[130]	train-logloss:0.227136	valid-logloss:0.242004
[140]	train-logloss:0.223158	valid-logloss:0.238846
[150]	train-logloss:0.219785	valid-logloss:0.23621
[160]	train-logloss:0.216907	

#### Parameter search 

In [81]:
clf = xgb.XGBClassifier(objective='binary:logistic')
#         eval_metric = 'logloss',
#         nthread = 16,
#         eta = 0.01,
#         num_boost_round = 4000,
#         max_depth = 9,
#         subsample = 0.75,
#         colsample_bytree = 1,
#         silent = 1
#         )

In [82]:
from scipy.stats import *

In [83]:
search_params = {
    'max_depth': randint(6, 10), 
    'learning_rate':[ 0.01,0.02,0.05],
    'n_estimators': randint(1500, 3500), 
                'gamma': uniform(), 
            'subsample':[0.75,0.85,0.95,1.0], 
            'colsample_bytree': [0.75,0.85,0.95,1.0], 
            'colsample_bylevel': [0.75,0.85,0.95,1.0],
            'reg_alpha': uniform(), 
            'reg_lambda': uniform(loc=1.0),
    
    }


In [84]:
from sklearn.model_selection import RandomizedSearchCV

In [85]:
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=search_params,
                                   n_iter=n_iter_search)

In [87]:
# random_search.fit(train_features,labels)

## Predictions as features 

In [42]:
kfold_test_pred = load_array("data/results/xgb_5fold_s2019_test_preds_maxcore.dat")
kfold_oof_pred  = load_array("data/results/xgb_5fold_s2019_oof_preds_maxcore.dat")

In [45]:
from scipy.special import logit


11.5729278828335

In [15]:
# train_features = np.hstack((train_features,kfold_oof_pred))
# test_features = np.hstack((test_features,kfold_test_pred))

In [46]:
train_features = np.hstack((train_features,logit(kfold_oof_pred)))
test_features = np.hstack((test_features,logit(kfold_test_pred)))

In [47]:
train_features.shape

(404290, 75)

## Randomized Stacking 

In [48]:
num_trials = 10
scores = []
models = []
params = []

In [71]:
def update_cache():
    global scores
    global models
    global params


    with open("data/cache/xgb_models_randomized.pik", 'w') as f:
        pickle.dump(models,f,protocol=pickle.HIGHEST_PROTOCOL)


    with open("data/cache/xgb_scores_randomized.pik", 'w') as f:
        pickle.dump(scores,f,protocol=pickle.HIGHEST_PROTOCOL)    


    with open("data/cache/xgb_params_randomized.pik", 'w') as f:
        pickle.dump(params,f,protocol=pickle.HIGHEST_PROTOCOL)        

In [72]:
update_cache()

In [2]:
def read_cache():
    
    global scores2
    global models2
    global params2
    
    with open("data/cache/xgb_models_randomized.pik", 'r') as f:
        models2 = pickle.load(f)	    

    with open("data/cache/xgb_scores_randomized.pik", 'r') as f:
        scores2 = pickle.load(f)	    

    with open("data/cache/xgb_params_randomized.pik", 'r') as f:
        params2 = pickle.load(f)	    
    


In [3]:
read_cache()

In [5]:
np.argmin(scores2)

16

In [7]:
params2[16]

{'colsample_bylevel': 0.9086627943049572,
 'colsample_bytree': 0.8662812861700129,
 'eval_metric': 'logloss',
 'gamma': 0.02933179779163947,
 'learning_rate': 0.028244194429306306,
 'max_depth': 9,
 'n_estimators': 3567,
 'objective': 'binary:logistic',
 'reg_alpha': 0.14600953461910307,
 'reg_lambda': 1.0750571669994455,
 'scale_pos_weight': 1,
 'subsample': 0.8097993636153147}

In [92]:
np.median(scores2)

0.20789099999999999

In [54]:

for i in range(num_trials):
    perm = np.random.permutation(len(train_features))
    ix_fit = perm[:int(len(train_features)*(0.9))]
    ix_valid = perm[int(len(train_features)*(0.9)):]
    y_val = labels[ix_valid]
    y_train = labels[ix_fit]
    
    d_train = xgb.DMatrix(train_features[ix_fit],label= y_train)#, weight=weight_train)
    d_valid = xgb.DMatrix(train_features[ix_valid],label= y_val)#,weight=weight_val)

        
    xgb_params = {
    'max_depth': np.random.randint(6, 10), 
    'learning_rate': 0.01 + np.random.rand() * 0.025,
    'n_estimators': np.random.randint(2000, 3500), 
    'objective': 'binary:logistic',
        'eval_metric' :'logloss',
        'scale_pos_weight': 1,
                'gamma': np.random.rand() * 0.1, 
            'subsample': 0.75+np.random.rand() * 0.25, 
            'colsample_bytree': 0.75+np.random.rand() * 0.25, 
            'colsample_bylevel': 0.75+np.random.rand() * 0.25,
            'reg_alpha': np.random.rand() * 0.2, 
            'reg_lambda': 1.0 + np.random.rand() * 0.3, 


    }
    
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    print xgb_params
    
    model = xgb.train(xgb_params, d_train, 6000,  watchlist, early_stopping_rounds=50, verbose_eval=10)#,feval=kappa)
    
    
    scores.append(model.best_score)
    models.append(model)
    params.append(xgb_params)
    
    print "best score and limit: ", model.best_score, model.best_ntree_limit
    
    with open("data/cache/xgb_models_randomized.pik", 'w') as f:
        pickle.dump(models,f,protocol=pickle.HIGHEST_PROTOCOL)
        
    
    with open("data/cache/xgb_scores_randomized.pik", 'w') as f:
        pickle.dump(scores,f,protocol=pickle.HIGHEST_PROTOCOL)    
    

{'reg_alpha': 0.050250743528123865, 'eval_metric': 'logloss', 'colsample_bylevel': 0.8849838276249586, 'scale_pos_weight': 1, 'learning_rate': 0.03180324929807718, 'n_estimators': 3431, 'colsample_bytree': 0.927309199143391, 'subsample': 0.8274389688474488, 'reg_lambda': 1.097588372463166, 'objective': 'binary:logistic', 'max_depth': 6, 'gamma': 0.024864051134649814}
[0]	train-logloss:0.672956	valid-logloss:0.673123
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 100 rounds.
[10]	train-logloss:0.527686	valid-logloss:0.529249
[20]	train-logloss:0.436503	valid-logloss:0.438873
[30]	train-logloss:0.378111	valid-logloss:0.381244
[40]	train-logloss:0.338489	valid-logloss:0.342191
[50]	train-logloss:0.310945	valid-logloss:0.3151
[60]	train-logloss:0.291502	valid-logloss:0.295921
[70]	train-logloss:0.277717	valid-logloss:0.282472
[80]	train-logloss:0.2675	valid-logloss:0.272491
[90]	train-logloss:0.259

[1480]	train-logloss:0.172829	valid-logloss:0.21154
[1490]	train-logloss:0.172617	valid-logloss:0.211524
[1500]	train-logloss:0.172368	valid-logloss:0.211489
[1510]	train-logloss:0.172109	valid-logloss:0.211441
[1520]	train-logloss:0.171884	valid-logloss:0.211415
[1530]	train-logloss:0.171678	valid-logloss:0.211396
[1540]	train-logloss:0.171424	valid-logloss:0.211367
[1550]	train-logloss:0.171206	valid-logloss:0.21134
[1560]	train-logloss:0.170981	valid-logloss:0.211321
[1570]	train-logloss:0.170781	valid-logloss:0.211311
[1580]	train-logloss:0.170511	valid-logloss:0.211278
[1590]	train-logloss:0.170274	valid-logloss:0.211244
[1600]	train-logloss:0.17005	valid-logloss:0.211197
[1610]	train-logloss:0.169839	valid-logloss:0.211183
[1620]	train-logloss:0.169647	valid-logloss:0.211167
[1630]	train-logloss:0.16941	valid-logloss:0.211126
[1640]	train-logloss:0.169195	valid-logloss:0.21111
[1650]	train-logloss:0.168945	valid-logloss:0.211081
[1660]	train-logloss:0.16875	valid-logloss:0.211058

best score and limit:  0.209544 2925
{'reg_alpha': 0.047868207526880216, 'eval_metric': 'logloss', 'colsample_bylevel': 0.8259820581740578, 'scale_pos_weight': 1, 'learning_rate': 0.01730446906395796, 'n_estimators': 2156, 'colsample_bytree': 0.969727756067095, 'subsample': 0.8844883849650509, 'reg_lambda': 1.0892437626103435, 'objective': 'binary:logistic', 'max_depth': 6, 'gamma': 0.019053895872791694}
[0]	train-logloss:0.682076	valid-logloss:0.682077
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 100 rounds.
[10]	train-logloss:0.589421	valid-logloss:0.589692
[20]	train-logloss:0.520691	valid-logloss:0.521097
[30]	train-logloss:0.468352	valid-logloss:0.46892
[40]	train-logloss:0.427129	valid-logloss:0.427874
[50]	train-logloss:0.394595	valid-logloss:0.395495
[60]	train-logloss:0.36834	valid-logloss:0.369318
[70]	train-logloss:0.347234	valid-logloss:0.348325
[80]	train-logloss:0.329675	valid-l

[1470]	train-logloss:0.192198	valid-logloss:0.210313
[1480]	train-logloss:0.19207	valid-logloss:0.210298
[1490]	train-logloss:0.191913	valid-logloss:0.210243
[1500]	train-logloss:0.191774	valid-logloss:0.210203
[1510]	train-logloss:0.191589	valid-logloss:0.210155
[1520]	train-logloss:0.191406	valid-logloss:0.210104
[1530]	train-logloss:0.191235	valid-logloss:0.210062
[1540]	train-logloss:0.191058	valid-logloss:0.210007
[1550]	train-logloss:0.190866	valid-logloss:0.209936
[1560]	train-logloss:0.190707	valid-logloss:0.20992
[1570]	train-logloss:0.190541	valid-logloss:0.209891
[1580]	train-logloss:0.190418	valid-logloss:0.209853
[1590]	train-logloss:0.190235	valid-logloss:0.209808
[1600]	train-logloss:0.19004	valid-logloss:0.209759
[1610]	train-logloss:0.189877	valid-logloss:0.209727
[1620]	train-logloss:0.189728	valid-logloss:0.209688
[1630]	train-logloss:0.189573	valid-logloss:0.20965
[1640]	train-logloss:0.189387	valid-logloss:0.209619
[1650]	train-logloss:0.189239	valid-logloss:0.2095

[3030]	train-logloss:0.170151	valid-logloss:0.206901
[3040]	train-logloss:0.170042	valid-logloss:0.206895
[3050]	train-logloss:0.169919	valid-logloss:0.206893
[3060]	train-logloss:0.169793	valid-logloss:0.206879
[3070]	train-logloss:0.169684	valid-logloss:0.206872
[3080]	train-logloss:0.169563	valid-logloss:0.206853
[3090]	train-logloss:0.169442	valid-logloss:0.206844
[3100]	train-logloss:0.169312	valid-logloss:0.206828
[3110]	train-logloss:0.169193	valid-logloss:0.206806
[3120]	train-logloss:0.16907	valid-logloss:0.206805
[3130]	train-logloss:0.168929	valid-logloss:0.206784
[3140]	train-logloss:0.168801	valid-logloss:0.206768
[3150]	train-logloss:0.168688	valid-logloss:0.206758
[3160]	train-logloss:0.168577	valid-logloss:0.206754
[3170]	train-logloss:0.168456	valid-logloss:0.206747
[3180]	train-logloss:0.168352	valid-logloss:0.206747
[3190]	train-logloss:0.168228	valid-logloss:0.20673
[3200]	train-logloss:0.168119	valid-logloss:0.206719
[3210]	train-logloss:0.16799	valid-logloss:0.206

[4590]	train-logloss:0.152907	valid-logloss:0.205632
[4600]	train-logloss:0.152798	valid-logloss:0.205622
[4610]	train-logloss:0.15271	valid-logloss:0.205631
[4620]	train-logloss:0.152611	valid-logloss:0.20562
[4630]	train-logloss:0.152498	valid-logloss:0.205621
[4640]	train-logloss:0.152388	valid-logloss:0.205621
[4650]	train-logloss:0.152285	valid-logloss:0.20561
[4660]	train-logloss:0.152189	valid-logloss:0.205605
[4670]	train-logloss:0.152093	valid-logloss:0.205599
[4680]	train-logloss:0.15199	valid-logloss:0.205592
[4690]	train-logloss:0.151884	valid-logloss:0.205591
[4700]	train-logloss:0.151803	valid-logloss:0.205598
[4710]	train-logloss:0.151693	valid-logloss:0.205585
[4720]	train-logloss:0.151591	valid-logloss:0.205576
[4730]	train-logloss:0.15149	valid-logloss:0.205577
[4740]	train-logloss:0.151391	valid-logloss:0.205569
[4750]	train-logloss:0.151291	valid-logloss:0.205552
[4760]	train-logloss:0.151178	valid-logloss:0.20555
[4770]	train-logloss:0.151064	valid-logloss:0.205551

[40]	train-logloss:0.330009	valid-logloss:0.332517
[50]	train-logloss:0.303756	valid-logloss:0.306566
[60]	train-logloss:0.285265	valid-logloss:0.288374
[70]	train-logloss:0.272427	valid-logloss:0.27603
[80]	train-logloss:0.263142	valid-logloss:0.267144
[90]	train-logloss:0.256033	valid-logloss:0.260426
[100]	train-logloss:0.250244	valid-logloss:0.254873
[110]	train-logloss:0.245823	valid-logloss:0.2508
[120]	train-logloss:0.242053	valid-logloss:0.247372
[130]	train-logloss:0.238855	valid-logloss:0.244513
[140]	train-logloss:0.236153	valid-logloss:0.242105
[150]	train-logloss:0.233865	valid-logloss:0.240074
[160]	train-logloss:0.231611	valid-logloss:0.238069
[170]	train-logloss:0.229412	valid-logloss:0.236165
[180]	train-logloss:0.227626	valid-logloss:0.234696
[190]	train-logloss:0.225808	valid-logloss:0.233255
[200]	train-logloss:0.224222	valid-logloss:0.231927
[210]	train-logloss:0.222881	valid-logloss:0.230875
[220]	train-logloss:0.2215	valid-logloss:0.229833
[230]	train-logloss:0.2

KeyboardInterrupt: 

#### prediction 

In [95]:
d_test = xgb.DMatrix(test_features)
preds = np.zeros((test_features.shape[0],1))
count = 0 

for m in tqdm_notebook(models2):
    if m.best_score <= np.median(scores2):
        p_test = m.predict(d_test)
        preds += p_test.reshape(-1,1)
        count +=1



In [96]:
preds_avg = preds / float(count) #devide by number of folds  

## KFold

In [29]:
xgb_params = {
'max_depth': 9, 
'learning_rate': 0.025,
'n_estimators': 3000, 
'objective': 'binary:logistic',
    'eval_metric' :'logloss',
    'scale_pos_weight': 1,
            'gamma': 0.3, 
        'subsample': 0.75, 
        'colsample_bytree': 0.75, 
        'colsample_bylevel': 1,
        'reg_alpha': 0.2, 
        'reg_lambda': 1, 
    

}


In [30]:
skf = StratifiedKFold(n_splits=num_folds, random_state=2019)
splits = skf.split(train_features, labels)


In [31]:
xgb_models = []

y_oof = np.zeros((train_features.shape[0],1))
preds = np.zeros((test_features.shape[0],1))

for ix_fit, ix_valid in tqdm_notebook(splits, total=num_folds):
    y_val = labels[ix_valid]
    y_train = labels[ix_fit]

                                           
    d_train = xgb.DMatrix(train_features[ix_fit],label= y_train)
    d_valid = xgb.DMatrix(train_features[ix_valid],label= y_val)
    
    
    
    ############## TRAINING
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    bst = xgb.train(xgb_params, d_train, 5000,  watchlist, early_stopping_rounds=50, verbose_eval=10)
    
    xgb_models.append(bst)

    ################ OOF 
    y_oof[ix_valid]= bst.predict(d_valid).reshape(-1,1)
#     xgb_models.append(bst)

    
    ################ Test prediction 
    d_test = xgb.DMatrix(test_features)
    p_test = bst.predict(d_test)
    preds += p_test.reshape(-1,1)
    
    

    


[0]	train-logloss:0.674834	valid-logloss:0.674969
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[10]	train-logloss:0.535424	valid-logloss:0.536969
[20]	train-logloss:0.443677	valid-logloss:0.446668
[30]	train-logloss:0.377698	valid-logloss:0.381988
[40]	train-logloss:0.330665	valid-logloss:0.336224
[50]	train-logloss:0.297188	valid-logloss:0.304028
[60]	train-logloss:0.272292	valid-logloss:0.280373
[70]	train-logloss:0.253311	valid-logloss:0.262566
[80]	train-logloss:0.239224	valid-logloss:0.249588
[90]	train-logloss:0.228449	valid-logloss:0.239908
[100]	train-logloss:0.22014	valid-logloss:0.2326
[110]	train-logloss:0.213535	valid-logloss:0.226934
[120]	train-logloss:0.208489	valid-logloss:0.22281
[130]	train-logloss:0.204427	valid-logloss:0.219574
[140]	train-logloss:0.201206	valid-logloss:0.217195
[150]	train-logloss:0.19869	valid-logloss:0.215473
[160]	train-logloss:0.196595	vali

[300]	train-logloss:0.185792	valid-logloss:0.209097
[310]	train-logloss:0.185299	valid-logloss:0.209078
[320]	train-logloss:0.184875	valid-logloss:0.20908
[330]	train-logloss:0.184461	valid-logloss:0.209077
[340]	train-logloss:0.183939	valid-logloss:0.209064
[350]	train-logloss:0.183445	valid-logloss:0.209074
[360]	train-logloss:0.182815	valid-logloss:0.209087
[370]	train-logloss:0.182189	valid-logloss:0.209101
[380]	train-logloss:0.181658	valid-logloss:0.209114
[390]	train-logloss:0.180912	valid-logloss:0.209141
[400]	train-logloss:0.18036	valid-logloss:0.209196
Stopping. Best iteration:
[354]	train-logloss:0.183216	valid-logloss:0.209062

[0]	train-logloss:0.674848	valid-logloss:0.674958
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[10]	train-logloss:0.533237	valid-logloss:0.534595
[20]	train-logloss:0.439826	valid-logloss:0.442368
[30]	train-logloss:0.375082	valid-logloss:0.3788

In [32]:
preds_avg = preds / float(num_folds) #devide by number of folds  

In [33]:
save_array("data/results/xgb_5fold_s2019_test_preds_wm_pseudo.dat",preds_avg)
save_array("data/results/xgb_5fold_s2019_oof_preds_wm_pseudo.dat",y_oof)

In [212]:
# plt.hist(preds_avg,bins=50)

In [214]:
# plt.hist(link_function(preds_avg),bins=50)

In [87]:
# def create_feature_map(features):
#     outfile = open('xgb.fmap', 'w')
#     i = 0
#     for feat in features:
#         outfile.write('{0}\t{1}\tq\n'.format(i, feat))
#         i = i + 1
#     outfile.close()
# xgb.plot_importance(bst)

In [215]:
# d_test = xgb.DMatrix(test_features)
# p_test = bst.predict(d_test)
# p_test = link_function(p_test)

In [34]:
p_test = link_function(preds_avg)

In [35]:
df_test = pd.read_csv(data_home+'test.csv')

sub = pd.DataFrame({'test_id': df_test['test_id'], 'is_duplicate': p_test.ravel()})
sub.head(2)

,is_duplicate,test_id
0,0.000057,0
1,0.092702,1


In [36]:
submission_name = "subm/xgb_wm_psudo.csv"

sub.to_csv(path+submission_name, index=False)
FileLink(submission_name)

/home/ubuntu/quora/subm/xgb_wm_psudo.csv

### Stacking 

In [69]:
from sklearn.ensemble import *

clfs = [
#         ExtraTreesClassifier(n_estimators=500, n_jobs=-1, criterion='gini',verbose=0),
# #     GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=9, n_estimators=500,verbose=1),
#         ExtraTreesClassifier(n_estimators=500, n_jobs=-1, criterion='entropy',verbose=0),
#         RandomForestClassifier(n_estimators=500, n_jobs=-1, criterion='gini',verbose=0),
        RandomForestClassifier(n_estimators=1000, n_jobs=-1, criterion='entropy',verbose=1, max_depth=9)
]

In [64]:
dataset_blend_train = np.zeros((train_features.shape[0], len(clfs)+1)) # +1 for xgboost results
dataset_blend_test = np.zeros((test_features.shape[0], len(clfs)+1))


In [65]:
num_folds = 5
X = train_features
X_submission = test_features
y = labels

skf = StratifiedKFold(n_splits=num_folds, random_state=2019)
skf  = list(skf.split(X, y))

In [66]:
np.any(np.isnan(X)), np.all(np.isfinite(X))

(False, True)

In [67]:
X = np.clip(X,X.min(),1e7)
X_submission = np.clip(X_submission,X_submission.min(),1e7)

In [11]:
# dataset_blend_train =   load_array("data/cache/dataset_blend_train_v1.dat")
# dataset_blend_test =    load_array("data/cache/dataset_blend_test_v1.dat")

In [ ]:
%%time 

for j, clf in enumerate(clfs):
    print j, clf
 
        
        
    dataset_blend_test_j = np.zeros((test_features.shape[0], num_folds))
    
    


    for i, (train, test) in enumerate(skf):
        print "Fold", i
        
        X_train = X[train]
        y_train = y[train]
        X_test = X[test]
        y_test = y[test]
        
#         print X_train.shape, y_train.shape, X_test.shape, y_test.shape
        clf.fit(X_train, y_train)
        
        y_submission = clf.predict_proba(X_test)[:, 1]
        dataset_blend_train[test, j] = y_submission
        dataset_blend_test_j[:, i] = clf.predict_proba(X_submission)[:, 1]
        
    dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)
    
    #caching to avoid dead-kernel interruptions 
    save_array("data/cache/dataset_blend_train_v1.dat",dataset_blend_train)
    save_array("data/cache/dataset_blend_test_v1.dat",dataset_blend_test)


0 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=9, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=-1, oob_score=False,
            random_state=None, verbose=1, warm_start=False)
Fold 0


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   37.6s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  3.6min finished
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    0.5s
[Parallel(n_jobs=16)]: Done 768 tasks      | elapsed:    1.0s
[Parallel(n_jobs=16)]: Done 1000 out of 1000 | elapsed:    1.3s finished
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.6s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    3.7s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    9.0s


In [13]:
dataset_blend_train.shape, dataset_blend_test.shape

((404290, 5), (2345796, 5))

In [45]:
dataset_blend_train[:,4]

array([  4.24634665e-02,   6.59618014e-03,   4.29204762e-01, ...,
         2.81577948e-02,   2.92460697e-07,   8.89144659e-01])

In [24]:
xgb_preds = load_array("data/results/xgb_5fold_s2019_test_preds_maxcore.dat")
xgb_y_oof = load_array("data/results/xgb_5fold_s2019_oof_preds_maxcore.dat")

In [34]:
dataset_blend_train[:,len(clfs)] = xgb_y_oof.ravel()
dataset_blend_test[:,len(clfs)] = xgb_preds.ravel()


In [32]:
len(clfs)

4

In [62]:
avg_blend_preds = dataset_blend_test.mean(axis=1)
avg_blend_preds[0:2]

array([  1.40080047e-04,   2.09554631e-01])

#### Blending

In [58]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(dataset_blend_train, y)
y_submission = clf.predict_proba(dataset_blend_test) [:, 1]

In [53]:
link_function(y_submission).max()

0.94942818288017461

In [57]:
# print "Linear stretch of predictions to [0,1]"
# y_submission = (y_submission - y_submission.min()) / (y_submission.max() - y_submission.min())

#### Submission

In [59]:
df_test = pd.read_csv(data_home+'test.csv')

sub = pd.DataFrame({'test_id': df_test['test_id'], 'is_duplicate': link_function(y_submission)})
sub.head(2)

,is_duplicate,test_id
0,0.008158,0
1,0.039296,1


In [60]:
submission_name = "subm/blendingv1.csv"

sub.to_csv(path+submission_name, index=False)
FileLink(submission_name)

/home/ubuntu/quora/subm/blendingv1.csv